<a href="https://colab.research.google.com/github/pixelpercebe/translated-videos-generator-tool/blob/main/video_translator_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalar los recursos necesarios ⬇


In [ ]:
!pip install git+https://github.com/openai/whisper.git moviepy
!pip install sacremoses
!pip install sacremoses
!pip install transformers
!pip install pydub

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# extraer el audio y generar la transcripcion mediante whisper ✍

In [ ]:
import whisper

# Cargar el modelo
modelo = whisper.load_model("large-v3")


In [ ]:
# Asegúrate de cambiar "audio_extraido.mp3" por tu archivo de audio
resultado = modelo.transcribe("audio_extraido.mp3", verbose=True)

In [ ]:
ruta_archivo = "transcripcion_videos.txt"
# Analizar los segmentos para identificar pausas y transcripción
with open(ruta_archivo, "w") as archivo:
  for segmento in resultado["segments"]:
    inicio = segmento["start"]
    fin = segmento["end"]
    texto = segmento["text"]
    archivo.write(f"Marca de tiempo: {inicio} a {fin}, Texto: {texto}")

#**Elegimos el idioma y generamos la traduccion del video.** 📔

In [ ]:
# @markdown ##Selecciona tu idioma:
# @markdown + en -> English
# @markdown + ara -> Arab
# @markdown + ru -> Russian
idioma = "ru" # @param ["en", "ara", "ru"]
from transformers import MarianMTModel, MarianTokenizer
# Determinar el modelo basado en el idioma seleccionado
print(f"Idioma seleccionado: {idioma}")
if idioma == 'ara':
    modelo_id = "Helsinki-NLP/opus-mt-es-ar"
    token_idioma = ">>ara<<"
elif idioma == 'en':
    modelo_id = "Helsinki-NLP/opus-mt-es-en"
    token_idioma = ">>en<<"
elif idioma == 'ru':
    modelo_id = "Helsinki-NLP/opus-mt-es-ru"
    token_idioma = ">>ru<<"

tokenizer = MarianTokenizer.from_pretrained(modelo_id)
model = MarianMTModel.from_pretrained(modelo_id)






In [ ]:
# Texto en español que deseas traducir
# Define el path al archivo de entrada
ruta_archivo_entrada = "transcripcion_videos.txt"
duracion_video = "Duracion.txt"

# Inicializa una variable para almacenar el texto modificado
texto_modificado = ""

# Abre el archivo de entrada para leer
with open(ruta_archivo_entrada, 'r', encoding='utf-8') as archivo_entrada:
    for linea in archivo_entrada:
        # Encuentra el inicio del texto después de "Texto: "
        inicio_texto = linea.find("Texto: ")
        if inicio_texto != -1:
            # Extrae solo el texto después de "Texto: "
            texto = linea[inicio_texto + 7:]  # 7 es la longitud de "Texto: "
            # Añade el token de identificación de idioma al principio del texto extraído
            texto_modificado += token_idioma + texto
            texto_antes = linea[:inicio_texto]
            with open(duracion_video, 'w', encoding='utf-8') as archivo_salida:
              archivo_salida.write(texto_antes)

# Traducción
entrada_tokenizada = tokenizer(texto_modificado, return_tensors="pt", padding=True)
traduccion_output = model.generate(**entrada_tokenizada)
texto_traducido = tokenizer.decode(traduccion_output[0], skip_special_tokens=True)

print(f"Texto traducido al {idioma}:", texto_traducido)

ruta_traduccion = "traduccion_video.txt"

with open(ruta_traduccion, "w") as archivo_tr:
  archivo_tr.write(texto_traducido)

#**Utilizamos la API de Google text to speech para generar voz.** 🔊

In [ ]:
ruta_archivo_traducido = "traduccion_video.txt"  # Cambia esto por la ruta real de tu archivo

with open(ruta_archivo_traducido, 'r', encoding='utf-8') as archivo:
    texto_traducido = archivo.read()


In [ ]:
from gtts import gTTS

# Convertir el texto en audio
if idioma == 'ara':
  idioma = 'ar'
tts = gTTS(text=texto_traducido, lang = idioma)  # Asegúrate de usar el código de idioma correcto ('es' para español, 'en' para inglés, etc.)

# Guardar el archivo de audio
ruta_archivo_audio = "audio_traducido.mp3"  # Puedes cambiar el nombre del archivo según necesites
tts.save(ruta_archivo_audio)


In [ ]:
from moviepy.editor import AudioFileClip
from pydub import AudioSegment
# Cargar el archivo de audio original
audio_original = AudioSegment.from_file("audio_traducido.mp3")

# Duración deseada en milisegundos (por ejemplo, para hacer que el audio tenga una duración de 2 minutos)
#TODO hay que cambiar esto para que obtenga la duracion guardada en la transcripcion del audio.
duracion_deseada_ms = 14.80 * 1000

# Calcular el factor de cambio de velocidad
factor_cambio = duracion_deseada_ms / len(audio_original)

# Ajustar la velocidad del audio (esto no cambia el pitch)
audio_ajustado = audio_original.speedup(playback_speed=factor_cambio)

# Guardar el audio ajustado
audio_ajustado.export("audio_traducido_ajustado.mp3", format="mp3")


In [ ]:

video_clip = VideoFileClip("/content/drive/MyDrive/videos/2024-03-11 14-58-12.mp4")
audio_clip = AudioFileClip("audio_traducido_ajustado.mp3")

In [ ]:
# Reemplazar el audio del video con el nuevo audio
# Esto asumirá que el audio debe comenzar al mismo tiempo que el video.
# Si el audio es más largo que el video, se cortará para que coincida con la duración del video.
video_final = video_clip.set_audio(audio_clip)

In [ ]:
# Exportar el video final
video_final.write_videofile("video_final.mp4")
